[Dataset Link](https://www.kaggle.com/c/fake-news/data?select=train.csv)

### Import Dependencies

In [55]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso,Ridge
from sklearn.metrics import accuracy_score

In [2]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\RISHABH
[nltk_data]     RATHORE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Pre-processing

In [4]:
#Loading the dataset
df = pd.read_csv('train.csv')

In [5]:
df.shape

(20800, 5)

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum() # counting the missing values

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.shape

(18285, 5)

In [10]:
# Combining Auther and news title
df['content'] = df['author'] +  ' ' + df['title']

In [11]:
df['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 18285, dtype: object

#### Splitting the dataset

In [12]:
X = df.drop(columns='label',axis=1)
Y = df['label']

#### Applying stemming

In [13]:
port_stem = PorterStemmer()

In [14]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return  stemmed_content

In [15]:
df['content'] = df['content'].apply(stemming)

In [16]:
df['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 18285, dtype: object

#### Agin Splitting the dataset 

In [17]:
X = df['content'].values
Y = df['label'].values

In [18]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [19]:
print(Y)

[1 0 1 ... 0 1 1]


#### Converting textual data to numerical data

In [20]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [21]:
print(X)

  (0, 14626)	0.2853880981846006
  (0, 12567)	0.25566372256502734
  (0, 8310)	0.3609049070394367
  (0, 8048)	0.29347549279156676
  (0, 7190)	0.24556189342497173
  (0, 6552)	0.21745594418933306
  (0, 4637)	0.23016077319140021
  (0, 3543)	0.2684494960336511
  (0, 3359)	0.3609049070394367
  (0, 2757)	0.2466340295002162
  (0, 2312)	0.3745612250433202
  (0, 247)	0.26982554594264346
  (1, 15663)	0.3053027963338981
  (1, 6377)	0.19285723710368197
  (1, 5140)	0.7119376870709988
  (1, 3328)	0.2623789770430963
  (1, 2619)	0.19368327535633711
  (1, 2066)	0.38191890436039194
  (1, 1764)	0.1509985164277699
  (1, 1391)	0.29617980713962144
  (2, 14560)	0.4180284001448272
  (2, 8973)	0.4948460479407663
  (2, 5579)	0.3490632212946542
  (2, 5031)	0.38709995799949964
  (2, 2895)	0.4581003415623782
  :	:
  (18282, 12239)	0.252743907968046
  (18282, 11515)	0.2748252773264482
  (18282, 11321)	0.24588400571511215
  (18282, 9605)	0.07665665104558947
  (18282, 8942)	0.1712955017712004
  (18282, 8879)	0.29296479

In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = .25,stratify= Y, random_state=2 )

In [27]:
X_train.shape

(13713, 15960)

In [28]:
X_test.shape

(4572, 15960)

In [29]:
Y_train.shape

(13713,)

In [30]:
Y_test.shape

(4572,)

### Traning and Testing  models

Evalution


### Linear Regression

In [49]:
LinrReg = LinearRegression()
LinrReg.fit(X_train,Y_train)
train_score_LinrReg  = LinrReg.score(X_train, Y_train)
test_score_LinrReg  = LinrReg.score(X_test, Y_test)
print("\nRidge Model............................................\n")
print("The train score for LinearRegression model is {}".format(train_score_LinrReg))
print("The test score for LinearRegression model is {}".format(test_score_LinrReg ))


Ridge Model............................................

The train score for LinearRegression model is 0.9913784787370168
The test score for LinearRegression model is 0.6058056718475064


### Ridge Regression

In [57]:
ridgeReg = Ridge(alpha=0.25)

ridgeReg.fit(X_train,Y_train)

#train and test scorefor ridge regression
train_score_ridge = ridgeReg.score(X_train,Y_train)
test_score_ridge = ridgeReg.score(X_test,Y_test)

print("\nRidge Model............................................\n")
print("The train score for ridge model is {}".format(train_score_ridge))
print("The test score for ridge model is {}".format(test_score_ridge))


Ridge Model............................................

The train score for ridge model is 0.9689095543233586
The test score for ridge model is 0.891456999434318


### LogisticRegression

In [44]:
LogReg = LogisticRegression()
LogReg.fit(X_train,Y_train)
train_score_LogReg  = LogReg.score(X_train, Y_train)
test_score_LogReg  = LogReg.score(X_test, Y_test)
print("\nLogisticRegression model............................................\n")
print("The train score for LogisticRegression model is {}".format(train_score_LogReg))
print("The test score for LogisticRegression model is {}".format(test_score_LogReg ))


LogisticRegression model............................................

The train score for LogisticRegression model is 0.9900094800554219
The test score for LogisticRegression model is 0.9811898512685914


## Conclusion : For the binary classification problem, logistic regression is one of the best models.
**The Linear Regression is also performing well on the training data, but we have the model that performed on unseen data, which means test data.**
 